In [ ]:
pwd


In [ ]:
!pip install optuna

In [ ]:
pip install seaborn

In [ ]:
import matplotlib.pyplot as plt
from pandas import DataFrame, concat
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dropout, Attention, MultiHeadAttention, Flatten
from tensorflow.keras.layers import Dropout, Dense, LSTM, Bidirectional, Attention, SimpleRNN, GRU
from tensorflow import keras
from tensorflow.keras.utils import plot_model
import optuna
import numpy as np
from numpy import concatenate
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from pylab import mpl
from datetime import datetime, timedelta

import math
import os
import time
import gc
import pathlib

# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

# display all columns of the dataframe
pd.options.display.max_columns = None

# display all rows of the dataframe
pd.options.display.max_rows = None

# to display the float values upto 6 decimal places
pd.options.display.float_format = '{:.6f}'.format


plt.rcParams['figure.figsize'] = (10, 7)


import seaborn as sns
sns.set_style('whitegrid')
# plt.style.use("fivethirtyeight")
%matplotlib inline

# 设置中文显示字体
mpl.rcParams["font.sans-serif"] = ["Arial"]

In [ ]:
def fillna_mean(df):
    for cols in list(df.columns[df.isnull().sum() > 0]):
        mean_val = df[cols].mean()  # 均值填充
        df[cols].fillna(mean_val, inplace=True)
    return df


def fillna_back(df):
    df = df.fillna(axis=0, method="bfill", limit=None)  #后一个数填充缺失值
    return df

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    """
     Frame a time series as a supervised learning dataset.
     Arguments:
     data: Sequence of observations as a list or NumPy array.
     n_in: Number of lag observations as input (X).
     n_out: Number of observations as output (y).
     dropnan: Boolean whether or not to drop rows with NaN values.
     Returns:
     Pandas DataFrame of series framed for supervised learning.
     """
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [ ]:
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
df_AQI = pd.DataFrame()
count = 0
for dirname, _, filenames in os.walk('C:/Users/DRDE-GWALIOR/Downloads/beijing'):
    # for dirname, _, filenames in os.walk('C:/Users/Balya/MTech/ML/PRESENTATIONS/PRSA_Data_20130301-20170228'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
#         path = dirname+'/'+ filename
        print('File path : ', path)
        if count == 0:
            df_AQI1 = pd.read_csv(path, encoding="utf-8")
            df_AQI = df_AQI.append(df_AQI1)
            count = 1
            continue

        df_AQI1 = pd.read_csv(path, encoding="utf-8")
        df_AQI = df_AQI.append(df_AQI1)

dataset = fillna_back(df_AQI)
dataset = dataset.drop(['No','year','month','day','hour','wd','station'], axis=1)

In [ ]:
#Remove outliers from the data set 

print('Before: Outliers Treating with Inter Quartile Range', dataset.shape)
df_outlier = dataset.copy()
# df_outlier.drop(['PM10'], axis=1, inplace=True)
df_outlier  = df_outlier.select_dtypes(exclude ='object')
#1st quantile
q1 = df_outlier.quantile(0.25)

#3rd quantile
q3 = df_outlier.quantile(0.75)

# IQR: The Interquartile Range is defined as the difference between the third and first quartile
iqr = q3-q1
# print(IQR)

df_outlier = df_outlier[~((df_outlier  < (q1-(1.5*iqr))) | (df_outlier  > (q3+(1.5*iqr)))).any(axis=1)]

# check the shape of the data
print('After: Outliers Treating with Inter Quartile Range', df_outlier.shape)

In [ ]:
df_outlier = df_outlier.select_dtypes(exclude='object').copy()
 
plt.figure(figsize = (15,5))

# ploting a boxplot to visualize the outliers in all the numeric variables
df_outlier.boxplot()

# setting plot label
# setting text size using 'fontsize'
plt.title('Distribution of all the Numeric Variables', fontsize = 15)

# xticks() returns the x-axis ticks
# 'rotation = vertical' rotates the x-axis labels vertically
plt.xticks(rotation = 'vertical', fontsize = 12)

# display the plot
plt.show()

In [ ]:
df_AQI.describe()

In [ ]:
dataset.describe()

In [ ]:
dataset.describe()

In [ ]:
# df_comp = df_comp.select_dtypes(exclude='object').copy()
plt.rcParams['figure.dpi'] = 100
df_out=df_outlier.copy()
outliers_by_variables = df_out.columns

plt.figure(figsize=(17,14))
for i in range(0,outliers_by_variables.shape[0]):
    plt.subplot(4,7, i+1)
    plt.boxplot(df_out[outliers_by_variables[i]], autorange=True)
    plt.title(outliers_by_variables[i], fontdict= {'fontsize': 15 ,'fontweight': 5, 'color':'000000'})
    plt.AutoLocator.default_params

In [ ]:
values = df_outlier.values
values = values.astype('float32')
scaler = MinMaxScaler()
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 3, 1)

In [ ]:
# split into train sets、validation sets and test sets
values = reframed.values
number=len(values)
n_train_hours = int(number*0.7)
n_valid_hours = int(number*0.9)
train = values[:n_train_hours, :]
valid = values[n_train_hours:n_valid_hours, :]
test = values[n_valid_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-11], train[:, -11:-5]
valid_X, valid_y = valid[:, :-11], valid[:, -11:-5]
test_X, test_y = test[:, :-11], test[:, -11:-5]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
valid_X = valid_X.reshape((valid_X.shape[0], 1, valid_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, valid_X.shape, valid_y.shape, test_X.shape,
      test_y.shape)  # [samples, timesteps, features]

In [ ]:
titles = [
    "PM2.5",
    "PM10",
    "SO2",
    "NO2",
    "CO",
    "O3",
     "TEM",
    "PRES",
    "DEMP",
    "RAIN",
    "WSPM",
]

feature_keys = [
    "PM2.5",
    "PM10",
    "SO2",
    "NO2",
    "CO",
    "O3",
     "TEM",
    "PRES",
    "DEMP",
    "RAIN",
    "WSPM",

]

colors = [
    "blue",
    "orange",
    "green",
    "red",
    "brown",
    "cyan",
    "gray",
    "olive",
    "purple",
    "pink",
    "gold",

]


In [ ]:
def visualize_loss(history, title):
    # Visualize training loss and validation loss
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure(dpi=300)
    plt.plot(epochs, loss,"green", label="Training loss")
    plt.plot(epochs, val_loss,color="purple", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


def visualize_acc(history, title):
    # Visualize training loss and validation loss
    acc = history.history["accuracy"]
    val_acc = history.history["val_accuracy"]
    epochs = range(len(acc))
    plt.figure(dpi=300)
    plt.plot(epochs, acc, "b", label="Training accuracy")
    plt.plot(epochs, val_acc, "r", label="Validation accuracy")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("accuracy")
    plt.legend()
    plt.show()

In [ ]:
def compare_visual(data1, data2):
    fig, axes = plt.subplots(
        nrows=2, ncols=3, figsize=(20, 15), dpi=300, facecolor="w", edgecolor="k"
    )
    for i in range(len(feature_keys)-5):
        key = feature_keys[i]
        ax = data1[key].plot(
            ax=axes[i // 3, i % 3],
            title="{}".format(titles[i]),
            rot=25,
            label="actual"
        )
        ax = data2[key].plot(
            ax=axes[i // 3, i % 3],
            color='red',
            rot=25,
            label='prediction'
        )
        ax.legend()
    plt.tight_layout()

    # scaler.inverse_transform()


def inv_scale(df, y):
    max = df.max()[-11:-5]
    min = df.min()[-11:-5]
#     df*(max-min)+min
    yy=y.copy()
    for i in range(len(yy)):
        yy[i] = yy[i]*(max-min)+min  
    return yy


def evaluate(model):
    # make a prediction
    yhat = model.predict(test_X)
    # move return_sequences=True
#     inv_yhat = scaler.inverse_transform(yhat)
    # invert scaling for actual
#     inv_y = scaler.inverse_transform(test_y)
    inv_yhat=inv_scale(dataset,yhat)
    inv_y=inv_scale(dataset,test_y)

    # calculate RMSE mean square error
#     aqi_inv_y = inv_y[:, -6:]
#     aqi_inv_yhat = inv_yhat[:, -6:]
    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    print('Test RMSE: %.3f' % rmse)
    mae = mean_absolute_error(inv_y, inv_yhat)
    print('Test MAE: %.3f' % mae)
    R2=r2_score(inv_y, inv_yhat)
    print('Test R2: %.3f' % R2)
    # Visualize true and predicted values
    y1 = DataFrame(
        inv_y[-100:], index=dataset.index[-100:], columns=dataset.columns[-11:-5])
    y2 = DataFrame(
        inv_yhat[-100:], index=dataset.index[-100:], columns=dataset.columns[-11:-5])
    compare_visual(y1,y2)

In [ ]:
epochs = 100
batch_size = 32
learning_rate = 0.001

timesteps = train_X.shape[1]
input_dim = train_X.shape[2]
output_dim=6

In [ ]:
# Define the input to the model, including time series data and labels
inputs = Input(shape=(timesteps, input_dim))


# Define RNN layer
rnn_out = SimpleRNN(128,activation='relu',return_sequences = False)(inputs)

# Define fully connected layer and output layer
dense = Dense(units=128, activation='relu')(rnn_out)

output = Dense(units=output_dim, activation='relu')(dense)

# Define the model and compile it
rnn = Model(inputs=inputs,  outputs=output)


rnn.compile(loss='mse', optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate), metrics=['accuracy'])


path_checkpoint = "model_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0,
    patience=10
)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
)

# fit network
history = rnn.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(valid_X, valid_y),
                     callbacks=[es_callback, modelckpt_callback]
                    )
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")

# evaluate(model)

In [ ]:
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")

In [ ]:
# Define the input to the model, including time series data and labels
inputs = Input(shape=(timesteps, input_dim))


# Define GRU Layer
gru_out = GRU(128, activation='relu', return_sequences=False)(inputs)

# Define fully connected layer and output layer
dense = Dense(units=128, activation='relu')(gru_out)

output = Dense(units=output_dim, activation='relu')(dense)

# Define the model and compile it
gru = Model(inputs=inputs,  outputs=output)


gru.compile(loss='mse', optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate), metrics=['accuracy'])



path_checkpoint = "model_checkpoint.h5"
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0,
    patience=10
)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
)

# fit network
history = gru.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(valid_X, valid_y),
                     callbacks=[es_callback, modelckpt_callback]
                    )
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")

# evaluate(model)

In [ ]:
inputs = Input(shape=(timesteps, input_dim))


# 定义LSTM层和多头注意力层
lstm_out = LSTM(128, activation='relu', return_sequences=False)(inputs)

# 定义全连接层和输出层
dense = Dense(units=128, activation='relu')(lstm_out)

output = Dense(units=output_dim, activation='relu')(dense)

# 定义模型并进行编译
lstm = Model(inputs=inputs,  outputs=output)


lstm.compile(loss='mse', optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate), metrics=['accuracy'])

# plot_model(model, to_file="Bi-LSTM-MultiheadAtt.png",
#            dpi=300, show_shapes=True)

path_checkpoint = "model_checkpoint_MultiAttLSTM_aqiPre.h5"
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0,
    patience=10
)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
)

# fit network
history = lstm.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(valid_X, valid_y),
#                     callbacks=[es_callback, modelckpt_callback]
                    )
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")

# evaluate(model)

In [ ]:
inputs = Input(shape=(timesteps, input_dim))



lstm_out = Bidirectional(
    LSTM(units=128, activation='relu', return_sequences=True))(inputs)
multihead_attn = MultiHeadAttention(
    num_heads=4, key_dim=32)(lstm_out, lstm_out)


flatten = Flatten()(multihead_attn)
dense = Dense(units=128, activation='relu')(flatten)

output = Dense(units=output_dim, activation='relu')(dense)


BiLSTMultiheadAtt = Model(inputs=inputs,  outputs=output)


BiLSTMultiheadAtt.compile(loss='mse', optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate), metrics=['accuracy'])




path_checkpoint = "model_checkpoint_MultiAttLSTM_aqiPre.h5"
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0,
    patience=10
)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
)


history = BiLSTMultiheadAtt.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(valid_X, valid_y),

                    )
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")


In [ ]:
#loss: 0.0056 - accuracy: 0.8156 - val_loss: 0.0065 - val_accuracy: 0.8058
#loss: 0.0058 - accuracy: 0.8138 - val_loss: 0.0066 - val_accuracy: 0.8003

In [ ]:
inputs = Input(shape=(timesteps, input_dim))



lstm_out = Bidirectional(
    LSTM(units=128, activation='relu', return_sequences=True))(inputs)
multihead_attn = MultiHeadAttention(
    num_heads=4, key_dim=32)(lstm_out, lstm_out)


flatten = Flatten()(multihead_attn)
dense = Dense(units=128, activation='relu')(flatten)

output = Dense(units=output_dim, activation='relu')(dense)


BiLSTMultiheadAtt = Model(inputs=inputs,  outputs=output)


BiLSTMultiheadAtt.compile(loss='mse', optimizer=keras.optimizers.Adam(
    learning_rate=learning_rate), metrics=['accuracy'])


path_checkpoint = "model_checkpoint_MultiAttLSTM_aqiPre.h5"
es_callback = keras.callbacks.EarlyStopping(
    monitor="val_loss", min_delta=0,
    patience=10
)

modelckpt_callback = keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=0,
    save_weights_only=True,
    save_best_only=True,
)


history = BiLSTMultiheadAtt.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(valid_X, valid_y),

                    )
visualize_loss(history, "Training and Validation Loss")
visualize_acc(history, "Training and Validation Accuracy")